In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
import parseIntervalFiles as pif
import parseActivityFiles as paf
#pun intended :)
import consolidateFiles as cf
import hervpd as hp

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [4]:
    verbose = True
    
    RAW_PATH = "/home/ju/GDrive/Projects/HeRV/Data/Raw"
    PRE_PATH = "/home/ju/GDrive/Projects/HeRV/Data/PreProcessed"
    
    # duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
    crop = 90
    
    # duration (in seconds) of each fragment to be sent to analysis
    duration = 240
    
    # if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
    threshold = 3
    

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [5]:
sessions = []
for user in range(7):
    sessions.extend(paf.get_user_sessions(user, dirname=RAW_PATH))

reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', '', '', '']
orphan start in: ['2017-10-14 11:58:31', 'start', 'focused-active', 'lying down', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171102.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171016.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171006.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171104.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171029.csv ... 
orphan stop in: ['2017-10-29 15:27:59', 'stop', '', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171028.csv ... 
orphan start in: ['2017-10-28 11:48:07', 'start', 'sleep', 'posture']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171013.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/

## 2 - Generate fragments from sessions

### 2.1 - breaks the sessions duration in fragments
Configurations:
* duration of each fragment in seconds;
* number of seconds to be discarded at the beginning of the session, accounting for user's stabilization and adjustment to posture and activity 

### 2.2 - extracts the intervals for each fragment
Retrieves from the heartbeat files all the intervals contained in each session's duration and adds them to the fragments objects (in memory)


In [6]:
frags = cf.fragment_sessions(sessions, duration, crop)

359 valid sessions out of 403 total (at least one full fragment of 240 seconds after discarding first 90 seconds)


## 3 - Quality assurance

### To be implemented


In [7]:
valid_frags = [f for f in frags if len(cf.beats_in_fragment(f, RAW_PATH)) > 0.8*duration ]

## 4 - Extract features

In [8]:
beats = cf.beatlist(cf.beats_in_fragment(frags[100], RAW_PATH))
print(cf.calc_metrics(beats))

{'rmssd': 246.14484218307996, 'sdnn': 206.23492014950645, 'nn50': 153, 'pnn50': 63.749999999999993, 'mrri': 1017.4066390041494, 'mhr': 61.104411490800594, 'total_power': 35554.907413156609, 'vlf': 4296.2253136113522, 'lf': 7408.2864799298786, 'hf': 23850.395619615378, 'lf_hf': 0.31061482576989419, 'lfnu': 23.699932250303196, 'hfnu': 76.300067749696794}


In [9]:
print(len(valid_frags))
df = [cf.aggregate_data(f, RAW_PATH) for f in valid_frags]

3458


In [10]:
pdf = pd.DataFrame(df)
pdf.head()

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,posture,rmssd,sdnn,sess,start,stop,total_power,user,vlf
0,focused-active,1823.334077,59.335074,1249.610728,0.685344,40.664926,74.326961,814.723333,100,0,33.444816,sitting,71.430042,81.293515,0,2017-10-14 02:15:51,2017-10-14 02:19:51,4837.108747,0,1764.163943
1,focused-active,406.541792,35.592981,735.654720,1.809543,64.407019,80.461096,756.130031,50,1,15.527950,sitting,37.668586,85.910298,0,2017-10-14 02:19:51,2017-10-14 02:23:51,4094.987842,0,2952.791329
2,focused-active,411.097626,54.835149,338.599659,0.823648,45.164851,75.682842,794.876623,28,2,9.120521,sitting,32.848284,41.129149,0,2017-10-14 02:23:51,2017-10-14 02:27:51,962.127780,0,212.430494
3,focused-active,759.792939,54.374329,637.544659,0.839103,45.625671,74.279914,811.800664,60,3,20.000000,sitting,43.694470,56.942077,0,2017-10-14 02:27:51,2017-10-14 02:31:51,2418.140880,0,1020.803282
4,focused-active,2873.564060,60.600346,1868.263744,0.650156,39.399654,73.898370,827.745763,114,4,38.775510,sitting,94.191584,119.360027,0,2017-10-14 02:31:51,2017-10-14 02:35:51,7640.073305,0,2898.245501


In [11]:
pdf.describe()

,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,rmssd,sdnn,sess,total_power,user,vlf
count,3.458000e+03,3458.000000,3.458000e+03,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3458.000000,3.458000e+03,3458.000000,3.458000e+03
mean,2.194103e+04,46.028661,2.504847e+05,2.426225,53.971339,78.695623,818.718038,71.373916,23.625795,25.645399,108.536266,111.676530,167.431463,6.092627e+05,1.616252,3.368371e+05
std,1.423297e+05,23.350799,3.178385e+06,4.956553,23.350799,18.303238,174.064760,57.659829,30.903771,22.827238,166.263260,121.070635,119.203970,8.755028e+06,1.993899,5.781785e+06
min,9.162266e-01,0.735464,2.168192e+00,0.020093,1.969717,47.660561,332.600543,0.000000,0.000000,0.000000,3.585467,5.939201,0.000000,9.926065e+00,0.000000,2.088845e+00
25%,3.549671e+02,26.249097,5.459688e+02,0.547914,35.396942,65.681984,703.113393,23.000000,2.000000,7.007826,32.477750,52.154415,66.000000,1.671809e+03,0.000000,4.636185e+02
50%,8.684488e+02,43.784665,1.249734e+03,1.283905,56.215335,77.051017,791.828479,58.000000,8.000000,18.782946,52.661276,75.209879,141.000000,3.443742e+03,1.000000,1.032988e+03
75%,3.792908e+03,64.603058,2.808493e+03,2.809655,73.750903,87.107640,933.909561,109.000000,35.000000,39.059440,121.260763,132.338163,297.000000,1.078053e+04,2.000000,2.386448e+03
max,3.900249e+06,98.030283,1.353277e+08,134.968592,99.264536,180.987972,1266.731959,306.000000,141.000000,90.666667,2205.670373,1826.309221,358.000000,3.308193e+08,6.000000,2.034236e+08


In [12]:
pdf.to_excel(PRE_PATH+'/'+'df_'+str(crop)+'_'+str(duration)+'.xlsx')

In [14]:
pdf.loc[pdf.hf > 1000000]

,activity,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,order,pnn50,posture,rmssd,sdnn,sess,start,stop,total_power,user,vlf
686,sleep,3.900249e+06,7.500135,4.810213e+07,12.333093,92.499865,83.008439,818.715719,120,61,40.268456,lie,492.320941,421.355980,44,2017-11-09 04:29:32,2017-11-09 04:33:32,7.137346e+07,0,1.937108e+07
707,sleep,1.667204e+06,3.479682,4.624533e+07,27.738262,96.520318,81.977972,935.591603,199,84,76.245211,lie,1212.083406,846.274598,44,2017-11-09 06:01:32,2017-11-09 06:05:32,1.020636e+08,0,5.415103e+07
1375,eat,1.469753e+06,20.680239,5.637286e+06,3.835534,79.319761,107.597365,960.885827,160,8,63.241107,sitting,1504.242866,1074.290610,111,2017-10-04 13:12:04,2017-10-04 13:16:04,8.263263e+06,0,1.156224e+06
1376,eat,1.112292e+06,20.145217,4.409076e+06,3.963957,79.854783,117.409066,774.337621,167,9,53.870968,sitting,1091.163104,806.968152,111,2017-10-04 13:16:04,2017-10-04 13:20:04,7.679258e+06,0,2.157890e+06
1377,eat,1.515957e+06,35.047890,2.809430e+06,1.853239,64.952110,106.594970,785.496795,157,10,50.482315,sitting,888.607522,698.812590,111,2017-10-04 13:20:04,2017-10-04 13:24:04,4.989375e+06,0,6.639891e+05
1456,focused-passive,1.304437e+06,29.769773,3.077313e+06,2.359112,70.230227,99.270507,908.841328,165,49,61.111111,sitting,1106.735287,888.700979,112,2017-10-05 12:52:13,2017-10-05 12:56:13,5.840989e+06,0,1.459239e+06
1551,leisure-passive,2.664805e+06,1.931123,1.353277e+08,50.783344,98.068877,118.899548,678.966006,150,11,42.613636,sitting,1584.140941,1145.269948,126,2017-09-29 20:04:28,2017-09-29 20:08:28,3.308193e+08,0,1.928268e+08
2287,not-recorded,1.200922e+06,4.482329,2.559144e+07,21.309828,95.517671,91.212019,828.928571,45,17,15.358362,sit,1036.693009,868.774555,235,2017-10-17 19:20:30,2017-10-17 19:24:30,4.144971e+07,2,1.465734e+07
2708,leisure-passive,1.028684e+06,3.044196,3.276297e+07,31.849392,96.955804,107.127956,651.116105,21,8,7.894737,lie,1155.731585,799.987282,316,2017-12-20 21:22:51,2017-12-20 21:26:51,5.969107e+07,4,2.589942e+07
2851,sleep,1.233466e+06,1.677667,7.228918e+07,58.606570,98.322333,94.480653,702.555556,12,79,3.934426,lie,1309.452236,938.945924,320,2017-12-21 05:45:49,2017-12-21 05:49:49,2.769463e+08,4,2.034236e+08
